In [1]:
import numpy as np
import matplotlib.pyplot as plt

# Constant

In [18]:
solar_constant = 1376  # Solar constant (W/m2)
albedo = 0.2  # over grass

L_v = 2257000  # specific latent heat of condensation of water (J/kg)
R_v = 461.57  # specific gas constant of water vapour (J/kg.K)

# Calculate the Net Radiative Flux reaching the surface

**input:**
<br>
long - Longitude
<br>
long - Longitude
<br>
t_utc - UTC time (hours)
<br>
day - Julian day of the year (day number e.g. 2nd Jan = 2)
<br>
<br>
**output:**
<br>
Q_s - Net radiative flux (W/m2)

In [16]:
def SolarFlux2(long, lat, t_utc, day, CFH, CFM, CFL):
    
    # convert day to decimal
    day = day + t_utc / 24  
    
    # ratio of distance between sun and earth
    d_ratio = 1 - 0.01672 * np.cos(0.9856 * (day - 4))
    
    # local hour
    local_h = (t_utc - 12) * np.pi / 12 + long * np.pi / 180
    
    # declination of the sun
    delta = (23.45 * np.pi / 180) * (np.cos((2 * np.pi * (day - 173)) / 365.25))

    # zenith
    zenith = np.sin(np.radians(lat)) * np.sin(delta) + np.cos(np.radians(lat)) * np.cos(delta) * np.cos(local_h)

    # calculate transmissivity
    T_K = (0.6 + 0.2 * np.sin(zenith)) * (1 - 0.4 * CFH) * (1 - 0.7 * CFM) * (1 - 0.4 * CFL)
    
    sin_zenith = np.sin(zenith)
    
    # Downwelling shortwave radiation transmitted through the air
    if sin_zenith < 0:
        k_s_down = 0 # nighttime
    else:
        k_s_down = solar_constant * T_K * sin_zenith 

    # Upwelling reflected short wave (solar) radiation
    K_s_up = -albedo * K_s_down

    # Net upward long wave radiation at the surface
    I_s_net = 0.08 * (1 - 0.1 * CFH - 0.3 * CFM - 0.6 * CFL) # 0.08 kms-1

    # Net radiation
    Q_s = K_s_up + K_s_down + I_s_net

    return Q_s

# Calculate the condensation (or evaporation) of water in air

**input:**
<br>
delta_T - Change in Temperature between current and previous time steps(K)
<br>
T - Current Temperature (K)
<br>
<br>
**output:**
<br>
deltaM - Mass of consensate/evaporation happening per m3 (g/m3)

In [23]:
def condensation(deltaT, T):
    # Constants for saturation vapor pressure formula
    c0 = 0.6105851e+03
    c1 = 0.4440316e+02
    c2 = 0.1430341e+01
    c3 = 0.2641412e-01
    c4 = 0.2995057e-03
    c5 = 0.2031998e-05
    c6 = 0.6936113e-08
    c7 = 0.2564861e-11
    c8 = -0.3704404e-13
    c9 = -4.8165754883E-17
    c10+1.3839187032E-18

    # Calculate saturation vapor pressure (esat, Pa) using the temperature (T)
    def saturation_vapor_pressure(T):
        # T-273.15 = C
        x = max(-80., T - 273.15)
        esat = c0+x*(c1+x*(c2+x*(c3+x*(c4+x*(c5+x*(c6+x*(c7+x*(c8+x*(c9+x*c10)))))))))
        return esat

    # Calculate saturation vapor pressure
    e_s = saturation_vapor_pressure(T)  # saturation vapour pressure (Pa)

    # Calculate mass of condensate/evaporation happening per m3 (g/m3)
    # Converts the result from kilograms per cubic meter to grams per cubic meter
    deltaM = -deltaT * ((L_v * e_s) / ((R_v ** 2) * (T ** 3))) * 1000
    
    return deltaM

# Initialize domain parameters

In [ ]:
domain = {
    "x_size": 5,
    "y_size": 5,
    "cell_size_t_1": 1 / 120,
    "cell_size_t_2": 1,
    "cell_size_d": 0.25,
    "t_size": 800,
    "d_x": None,
    "d_y": None,
    "d_z": None,
    "z_size": None,
}

fog_domain = {}

domain["d_x"] = domain["x_size"] / domain["cell_size_d"]
domain["d_y"] = domain["y_size"] / domain["cell_size_d"]
domain["d_z"] = len(np.arange(0.93, 1.05, 0.001 * domain["cell_size_d"]))
domain["z_size"] = domain["d_z"] * domain["cell_size_d"]
wind_increase = 1
vert_temp_multipliers = np.arange(0.93, 1.05, 0.001 * domain["cell_size_d"])

# Initialize initial condition parameters

In [ ]:
initialCond = {
    "Temp_t0_C": 12.3,
    "Vert_temp_profileC": None,
    "soil_tempC": 10,
    "P": 101325,
    "t_utc": 11,
    "RH_0": 1,
    "T_dpC": 12.3,
    "wind_speed": 0.1,
    "wind_direction": 49,
    "R_0": 1,
}


initialCond["Vert_temp_profileC"] = initialCond["Temp_t0_C"] * np.arange(0.93, 1.05, 0.001 * domain["cell_size_d"])

# Define a dictionary for wind vectors calculations
windVectors = {
    "wind_u": None,
    "wind_v": None,
    "wind_w": 0,
    
}

# Calculate wind vectors

In [ ]:
windVectors = {
    "wind_u": None,
    "wind_v": None,
    "wind_w": 0,
    
}

windVectors["wind_u"] = initialCond["wind_speed"] * np.cos(np.radians(initialCond["wind_direction"]))
windVectors["wind_v"] = initialCond["wind_speed"] * np.sin(np.radians(initialCond["wind_direction"]))
windVectors["wind_w"] = 0